Rodzaje ruchu:\
Swap - losujemy dwa miasta i zamieniamy je ze sobą.\
Insercja - losujemy miasto i punkt w którym je umieszczamy.\
Zamiana kolejności - losujemy dwa punkty i między nimi zamieniamy kolejność.\
\
Sąsiedztwo to dwa przypadki które różnią się między sobą jednym ruchem\
\
Klasyczny algorytm wspinaczki z multistartem: po każdym ruchu liczymy drogę i sprawdzamy czy jest mniejsza czy nie, wybieramy tylko lepsze opcje. Multistart polega na wielokrotnym odpaleniu algorytmu z losowaniem punktu startowego.\
\
Algorytm symulowanego wyżarzania: zmodyfikowany algorytm wspinaczki, jeśli znalezione rozwiązanie jest gorsze to nadal je rozważamy, na podstawie tego jak bardzo jest gorsze oraz od temperatury.\
Temperatura jest wysoka na początku działania algorytmu i spada razem z czasem wykonywania algorytmu, im niższa temperatura tym mniejsze prawdopodobieństwo wybrania gorszego rozwiązania, bo algorytm już trochę działa i zakładamy że jest w lepszym miejscu niż na początku.\
Pozwala on nam wyjść z niektórych ekstremów lokalnych dzięki możliwości przyjęcia gorszego rozwiązania.\
\
Tabu search:\
Sąsiedztwo wygląda tutaj inaczej: przy swapie losujemy jedną liczbę i zamieniamy wszystkie kombinacje z tą liczbą, ze wszystkich kombinacji wybieramy najlepszą.\
Następnie taka kombinacja trafia na listę tabu, czyli takiej samej zamiany nie można wykonać przez określoną kolejną liczbę ruchów.\
\
Funkcja aspiracji to funkcja, która pod pewnymi warunkami pozwala wykonać ruch z listy tabu ale nie trzeba tego implementować.\
\
Odpowiednia długość listy tabu pozwala wyjść z niektórych ekstremów lokalnych, jeśli będzie za krótka to możemy nie wyjść z ekstremum lokalnego, jeśli będzie za długa to możemy wyjść z ekstremum niepotrzebnie.


In [1]:
import numpy as np
import pandas as pd
import math
import random

In [2]:
#wspinaczka z multistartem
distance = np.array([[0,2,4,1,5],
                    [2,0,3,8,7],
                    [4,3,0,5,1],
                    [1,8,5,0,3],
                    [5,7,1,3,0]])

In [3]:
def Odleglosc(distance, trasa):
    suma = 0
    n = len(trasa)
    
    for i, current_city in enumerate(trasa):
        next_city = trasa[(i + 1) % n]  # Use modulo to handle the last element
        suma += distance[current_city][next_city]

    return suma

In [4]:
#swap
def Swap(trasa):
    trasa_temp = trasa.copy()
    #print(trasa)
    swap = np.random.choice(trasa_temp,2,replace=False)
    temp = trasa_temp[swap[0]]
    trasa_temp[swap[0]] = trasa_temp[swap[1]]
    trasa_temp[swap[1]] = temp
    #print(swap)
    #print(trasa)
    return trasa_temp

#insercja
def Insercja(trasa):
    trasa_temp = trasa.copy()
    insercja = np.random.choice(trasa_temp,2,replace=False)
    #print(trasa_temp)
    #print(insercja)
    temp = trasa_temp[insercja[0]]
    trasa_temp = np.delete(trasa_temp,insercja[0])
    trasa_temp = np.insert(trasa_temp,insercja[1],temp)
    #print(trasa_temp)
    return trasa_temp

def Zamiana(trasa):
    trasa_temp = trasa.copy()
    zamiana = np.random.choice(trasa_temp,2,replace=False)
    zamiana.sort()
    #print(trasa_temp)
    #print(zamiana)
    for i in range(math.ceil((zamiana[1]-zamiana[0])/2)):
        temp = trasa_temp[zamiana[1]]
        trasa_temp[zamiana[1]] = trasa_temp[zamiana[0]]
        trasa_temp[zamiana[0]] = temp
        zamiana[1] -= 1
        zamiana[0] += 1
    #print(trasa_temp)
    return trasa_temp

In [5]:
#Zamiana(trasa)

In [6]:
#trasa = np.arange(len(distance))
#np.random.shuffle(trasa)
#Insercja(trasa)

In [7]:
#temp = trasa[0] #temp = 4
#trasa = np.delete(trasa,0)
#trasa = np.insert(trasa,4,3)

In [8]:
#ruch polegający na swapie, sprawdzenie sąsiadów, zwrot mniejszej trasy
def Ruch_swap(distance,trasa):
    odl = Odleglosc(distance,trasa)
    trasa_new = Swap(trasa)
    odl_new = Odleglosc(distance,trasa_new)
    if odl_new < odl:
        return trasa_new, odl_new
    return trasa, odl, trasa_new, odl_new

#ruch polegający na insercji, sprawdzenie sąsiadów, zwrot mniejszej trasy
def Ruch_insercja(distance,trasa):
    odl = Odleglosc(distance,trasa)
    trasa_new = Insercja(trasa)
    odl_new = Odleglosc(distance,trasa_new)
    if odl_new < odl:
        return trasa_new, odl_new
    return trasa, odl

#ruch polegający na zamianie, sprawdzenie sąsiadów, zwrot mniejszej trasy
def Ruch_zamiana(distance,trasa):
    odl = Odleglosc(distance,trasa)
    trasa_new = Zamiana(trasa)
    odl_new = Odleglosc(distance,trasa_new)
    if odl_new < odl:
        return trasa_new, odl_new  
    return trasa, odl

In [9]:
distance = pd.read_excel("Dane_TSP_48.xlsx")
distance = distance.to_numpy()
distance = np.delete(distance,0,1)
distance

array([[   0, 1495,  381, ...,  488,  753, 1184],
       [1495,    0, 1135, ..., 1727, 1388,  661],
       [ 381, 1135,    0, ...,  640,  591,  810],
       ...,
       [ 488, 1727,  640, ...,    0,  520, 1242],
       [ 753, 1388,  591, ...,  520,    0,  801],
       [1184,  661,  810, ..., 1242,  801,    0]], dtype=int64)

In [10]:
def IHC(distance, ruch, n_iter_ruch, n_iter):  # ruch = {Swap, Insercja, Zamiana}
    min_iter = []
    for i in range(n_iter):
        trasa = np.arange(len(distance))
        np.random.shuffle(trasa)
        odl = Odleglosc(distance,trasa)
        for j in range(n_iter_ruch):
                trasa_new = ruch(trasa)
                odl_new = Odleglosc(distance,trasa_new)
                if odl_new < odl:
                    odl = odl_new
                    trasa = trasa_new
        min_iter.append((i,trasa,odl))
    return min_iter
wynik = IHC(distance,Swap,1000,100)
wynik.sort(key=lambda x: x[2])
wynik[0:10]



[(76,
  array([36, 18, 16, 26, 29, 42,  5, 17, 37, 45, 14, 10, 22, 28,  1,  3, 25,
         44, 34,  9, 23, 41, 47,  4, 38, 31, 20, 46, 19, 11, 13, 24, 40, 21,
          0,  7,  8, 15,  2, 33, 12, 39, 30, 43, 35, 32, 27,  6]),
  14563),
 (58,
  array([ 3,  1, 41,  4, 47, 28, 22,  2, 33, 40, 15,  8,  7, 21, 13, 24, 38,
         12, 46, 14, 37, 30, 45, 32, 19, 27,  5, 36, 18, 42, 26, 16, 29,  6,
         35, 43, 17, 11,  0, 39, 10, 20, 31,  9, 44, 34, 23, 25]),
  15347),
 (78,
  array([14, 32, 35,  6, 17, 45,  8, 21,  1, 25,  3,  9, 41, 44, 34, 23, 31,
         38, 20, 47,  4, 28, 40, 15, 22, 10, 12, 13, 24, 33,  2, 39, 27, 29,
         42, 36, 16, 26, 18,  5,  0,  7, 37, 30, 43, 19, 46, 11]),
  15728),
 (71,
  array([ 1, 13, 46, 19, 32, 43,  6, 17, 35, 45, 30, 36,  5, 26, 18, 16, 42,
         29, 27, 20,  4, 47, 24, 38, 23, 34, 44,  9, 25,  3, 28, 33, 40, 15,
          0, 37, 14, 10, 11, 39,  8,  7, 21,  2, 22, 12, 31, 41]),
  16129),
 (94,
  array([ 9, 34, 44, 23, 47, 41,  1,  3, 25, 2

In [11]:
def Geometric(T,alpha):
    return alpha*T

def Slow(T,alpha):
    return T/(1+alpha*T)


In [12]:
def SA(distance, ruch, n_iter_ruch, n_iter, T_start, cooling, alpha):  # ruch = {Swap, Insercja, Zamiana}, cooling = {Geometric, Slow}, alpha = cooling param 
    min_iter = []
    for i in range(n_iter):
        T = T_start
        trasa = np.arange(len(distance))
        np.random.shuffle(trasa)
        odl = Odleglosc(distance,trasa)
        for j in range(n_iter_ruch):
                trasa_new = ruch(trasa)
                odl_new = Odleglosc(distance,trasa_new)
                diff_odl = odl_new - odl
                if diff_odl < 0:
                    odl = odl_new
                    trasa = trasa_new
                elif random.random() < math.exp(-diff_odl/T):
                    odl = odl_new
                    trasa = trasa_new
                T = cooling(T,alpha)
        min_iter.append((i,trasa,odl))
    return min_iter
wynik = SA(distance,Swap,1000,100,100,Slow,0.1)
wynik.sort(key=lambda x: x[2])
wynik[0:10]

[(20,
  array([ 9,  3, 41, 28,  4, 33, 13, 12, 20, 31, 47, 23, 34, 44, 25,  1, 40,
         15,  0,  8, 26, 16, 42, 43, 17, 35,  6, 27, 29,  5, 18, 36, 30, 37,
          7, 21,  2, 22, 19, 32, 45, 14, 39, 11, 46, 10, 24, 38]),
  15706),
 (4,
  array([ 8,  7, 21,  0, 28,  4, 13, 31, 20, 24, 38, 47, 41,  9, 23, 44, 34,
          3, 25,  1, 33, 15, 40, 22, 12,  2, 39, 11, 10, 46, 45, 30, 43, 35,
         27, 29, 19, 14, 32,  5, 26, 42, 16, 18, 36,  6, 17, 37]),
  15976),
 (35,
  array([20, 31, 23, 44, 34,  9, 41, 47, 33, 22, 13, 12, 38, 24, 46,  2, 21,
         37, 30, 36, 26, 29, 19,  4, 28, 25,  3,  1, 40, 15,  0,  7,  8, 39,
         10, 11, 32, 14, 43, 35,  6, 17, 27,  5, 18, 42, 16, 45]),
  16692),
 (10,
  array([18, 26, 16, 42, 35, 17, 30,  8, 47, 23,  9, 25,  3, 34, 44, 41,  1,
          4, 12, 24, 38, 31, 20, 22, 39,  2, 33, 28, 13, 10, 32,  5, 36, 29,
         46, 19, 11, 14,  0, 40, 15, 21,  7, 37, 43, 45,  6, 27]),
  16733),
 (88,
  array([ 3,  1, 28, 13, 12, 22, 33, 40, 15, 21

In [13]:
# wersja SA gdzie warunkiem stopu jest temperatura a nie liczba iteracji 
# (chyba lepiej tak bo upewniamy sie ze temperatura jest niska a podajac liczbe iteracji mozemy skonczyc na wysokiej temperaturze)
# mozna tez dodac warunek stopu ze po np 100 iteracjach bez poprawy rozwiazania to koniec

def SA(distance, ruch, n_iter, T_start, T_stop, cooling, alpha):  # ruch = {Swap, Insercja, Zamiana}, cooling = {Geometric, Slow}, alpha = cooling param 
    min_iter = []
    for i in range(n_iter):
        T = T_start
        trasa = np.arange(len(distance))
        np.random.shuffle(trasa)
        odl = Odleglosc(distance,trasa)
        while(T > T_stop):
                trasa_new = ruch(trasa)
                odl_new = Odleglosc(distance,trasa_new)
                diff_odl = odl_new - odl
                if diff_odl < 0:
                    odl = odl_new
                    trasa = trasa_new
                elif random.random() < math.exp(-diff_odl/T):
                    odl = odl_new
                    trasa = trasa_new
                T = cooling(T,alpha)
        min_iter.append((i,trasa,odl))
    return min_iter
wynik = SA(distance,Swap,100,100,0.01,Slow,0.1)
wynik.sort(key=lambda x: x[2])
wynik[0:10]

[(38,
  array([ 7, 30, 37, 43, 27,  6,  5, 36, 35, 45, 11, 39,  8, 14, 19, 20, 10,
         29, 42, 16, 26, 18, 17, 32, 46, 24, 41, 23, 44, 34, 25,  3,  1,  9,
         28, 40,  4, 47, 31, 38, 12,  2, 33, 13, 22, 15, 21,  0]),
  16555),
 (16,
  array([ 7, 43, 17, 14, 37,  8,  0, 39, 11, 32, 30, 45, 27, 29, 18, 36, 19,
         20, 31, 38, 47,  4, 22, 10, 46, 12, 24, 13,  2, 35,  5, 42, 16, 26,
          6, 15, 40,  1, 41, 23,  9, 34, 44, 25,  3, 28, 33, 21]),
  16849),
 (85,
  array([ 6,  5, 36, 26, 18, 42, 16, 14, 21, 40, 15,  8, 39, 22, 10,  2, 12,
         20, 46, 11, 19, 32, 45, 35, 27, 29, 31,  4, 47, 28, 41,  9, 25,  3,
          1, 33, 13, 24, 38, 23, 34, 44,  0,  7, 37, 30, 43, 17]),
  17089),
 (45,
  array([ 5, 36, 29, 26, 16, 18, 27, 42, 45, 38, 23,  1, 28,  4, 47, 24, 10,
         22, 12, 13, 33, 40,  2, 21, 15, 39,  8,  0, 34, 44,  9, 25,  3, 41,
         31, 20, 46,  6, 43, 30, 37,  7, 14, 11, 32, 19, 17, 35]),
  17092),
 (69,
  array([17, 32, 45, 40,  2, 33,  4, 34, 44, 4

In [14]:
# funkcje pomocnicze do sprawdzenia ile iteracji odpowiada parametrowi T_stop
def T_stopToIter(T_start, T_stop, cooling, alpha):
    T = T_start
    i = 0
    while(T > T_stop):
        T = cooling(T,alpha)
        i += 1
    return i
def IterToT_stop(T_start, n_iter, cooling, alpha):
    T = T_start
    for i in range(n_iter):
        T = cooling(T,alpha)
    return T
print(T_stopToIter(100,0.01,Slow,0.1))
IterToT_stop(100,100,Slow,0.1)

1000


0.09990009990009993

In [15]:
# #algorytm ze swapem
# trasa = np.arange(len(distance))
# np.random.shuffle(trasa)
# licza_iteracji = 5

# print("Start")
# print(trasa)
# print(Odleglosc(distance,trasa))
# for i in range(licza_iteracji):
#     print("\n{} iteracja".format(i+1))
#     trasa = Ruch_swap(distance,trasa)
#     print(trasa)
#     print(Odleglosc(distance,trasa))


In [16]:
# #algorytm z insercją
# trasa = np.arange(len(distance))
# np.random.shuffle(trasa)
# licza_iteracji = 5

# print("Start")
# print(trasa)
# print(Odleglosc(distance,trasa))
# for i in range(licza_iteracji):
#     print("\n{} iteracja".format(i+1))
#     trasa = Ruch_insercja(distance,trasa)
#     print(trasa)
#     print(Odleglosc(distance,trasa))

In [17]:
# #algorytm z zamianą
# trasa = np.arange(len(distance))
# np.random.shuffle(trasa)
# licza_iteracji = 1000

# print("Start")
# print(trasa)
# print(Odleglosc(distance,trasa))
# for i in range(licza_iteracji):
#     print("\n{} iteracja".format(i+1))
#     trasa = Ruch_zamiana(distance,trasa)
#     print(trasa)
#     print(Odleglosc(distance,trasa))

In [20]:
#NN
min_odl = []
for n in range(len(distance)):
    potencjalne = np.arange(len(distance))
    trasa = np.zeros(len(distance),dtype=int)
    trasa[0] = potencjalne[n]
    potencjalne = np.delete(potencjalne,int(trasa[0]))

    for cur in range(1,len(trasa)):
        odleglosci = []
        for i in range(len(potencjalne)):
            trasa[cur] = potencjalne[i]
            odleglosci.append((potencjalne[i],Odleglosc(distance,trasa[0:cur+1])))
        odleglosci.sort(key=lambda x: x[1])
        trasa[cur] = odleglosci[0][0]
        potencjalne = np.delete(potencjalne,potencjalne==trasa[cur])
    min_odl.append((Odleglosc(distance,trasa),trasa))
    
min_odl.sort(key=lambda x:x[0])
min_odl[:10]